In [72]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException
import time
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd
import os
import re
from functions import allMondays, intHolidayClosures, create_trips, country_holidays

In [75]:
#constants
URL = 'https://camper.fti.de/'
DELAY = 3
PAGE_LOAD_ID = "SearchMask"
COOKIE_ID = '//*[@id="uc-center-container"]/div[2]/div/div[1]/div/button'
COOKIE_JS_PATH = 'document.querySelector("#usercentrics-root").shadowRoot.querySelector("#uc-center-container > div.sc-gWXbKe.fSIThp > div > div.sc-cxpSdN.ihpUDX > div > button")'
NOTIFICATION_JS_PATH = 'document.querySelector("#pid-14572 > div.cleverpush-confirm.cleverpush-confirm-modal.cleverpush-confirm-windows.cleverpush-confirm-chrome.cleverpush-confirm-chrome.cleverpush-confirm-1 > div.cleverpush-confirm-wrap > div.cleverpush-clearfix.cleverpush-clearfix-bottom > div.cleverpush-confirm-buttons > button.cleverpush-confirm-btn.cleverpush-confirm-btn-deny")'
POPUP_JS_PATH = 'document.querySelector("#trboModule_12920_202268_88_container > div.trbo-closeButton.trbo-theme-0.trboType-icon > i")'
CITY = 'Los Angeles'
CITY_JS_PATH = 'document.querySelector("#SearchMask > div.flexContainer.location > div:nth-child(1) > div:nth-child(1) > div > div > input")'
CAL_ELEMS_XPATH = "//span[contains(@class, 'cell day')]"
PU_CALENDAR_JS_PATH = 'document.querySelector("#SearchMask > div:nth-child(2) > div:nth-child(1) > div > div > div:nth-child(1) > input")'
DO_CALENDAR_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(1) > input')"
DO_MONTH_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > header > span.day__month_btn.up')"
PU_MONTH_TEXT_CLASS_NAME = "vdp-datepicker__calendar"
PU_MONTH_NEXT_BTN_CLASS_NAME = "span[class = 'next']"
DO_MONTH_TEXT_XPATH = "//*[@id='SearchMask']/div[2]/div[2]/div/div/div[2]/header/span[3]"
SUBMIT_BUTTON_ID = "SubmitButton"
LOAD_MORE_RESULTS = "document.querySelector('#ResultList > button')"
# PICKUP_LOC_ID = 'pickUpLocation'
# PICKUP_LOC_XPATH = "//select[@id = 'pickUpLocation]"

In [4]:
def create_browser(URL, ELEMENT_ID):

    """Creates the browser and loads the page given url and webelement"""
    
    s=Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=s)

    browser.get(URL)

    try:
        myElem = WebDriverWait(browser, DELAY).until(EC.presence_of_element_located((By.ID, ELEMENT_ID)))
        print ("Page is ready!")
    except TimeoutException:
        print ("Couldn't find element")

    return browser


browser = create_browser(URL, PAGE_LOAD_ID)

Page is ready!


In [5]:
#accept cookies
browser.execute_script(f"return {COOKIE_JS_PATH}").click()

In [7]:
#deny news
browser.execute_script(f"return {NOTIFICATION_JS_PATH}").click()

In [ ]:
#close popup
browser.execute_script(f"return {POPUP_JS_PATH}").click()

In [6]:
#use if running code below more than once

browser.execute_script(f"return {CITY_JS_PATH}").click()
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").clear()

In [8]:
browser.execute_script(f"return {CITY_JS_PATH}").send_keys(f'{CITY}')
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").send_keys(Keys.RETURN)

In [54]:
#simulate a single trip
german_months = {
    1 : 'Jan',
    2 : 'Feb',
    3 : 'Mär',
    4 : 'Apr',
    5 : 'Mai',
    6 : 'Jun',
    7 : 'Jul',
    8 : 'Aug',
    9 : 'Sep',
    10: 'Okt',
    11: 'Nov',
    12: 'Dez'

}

start_date = date(2022, 11, 14)
end_date = start_date + timedelta(17)
trip = {}
trip['start_date'] = start_date
trip['start_month'] = german_months[start_date.month]
trip['start_day'] = start_date.day
trip['end_date'] = end_date
trip['end_day'] = end_date.day
trip['end_month'] = german_months[end_date.month]
trip['end_day']
trip['city'] = CITY

In [30]:
#open the pickup month calendar
browser.execute_script(f"return {PU_CALENDAR_JS_PATH}").click()

In [31]:
#select the pickup month
cal_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]

while cal_month != trip['start_month']:
    
    browser.find_element(By.CSS_SELECTOR, f"{PU_MONTH_NEXT_BTN_CLASS_NAME }").click()
    time.sleep(1)
    cal_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]
    print(cal_month)
    

Nov


In [32]:
#click on the pickup day
cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
cal_elems_dict = {}
for ele in cal_elems:

    if ele.text.isdigit():

        cal_elems_dict[ele.text] = ele
        
cal_elems_dict[str(trip['start_day'])].click()

In [36]:
#open the drop off calendar

browser.execute_script(f"return {DO_CALENDAR_JS_PATH}").click()

In [37]:
#select the drop off month
cal_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
while cal_month != trip['end_month']:
    browser.find_element(By.XPATH, f"{DO_MONTH_TEXT_XPATH}").click()
    #updates cal_month variable to whatever date you are currently on
    time.sleep(int(np.random.rand(1)*3))
    cal_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
    print(cal_month)

Dez


In [38]:
#select drop off day
cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
cal_elems_dict = {}
for ele in cal_elems:

    if ele.text.isdigit():

        cal_elems_dict[ele.text] = ele
        
cal_elems_dict[str(trip['end_day'])].click()

In [39]:
#click search
browser.find_element(By.ID, f"{SUBMIT_BUTTON_ID }").click()

In [40]:
#ADD LOGIC TO WAIT FOR PAGE TO LOAD AND GO BACK IF THERE ARE NO RESULTS

In [ ]:
#load all the results
browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()

In [77]:
LISTINGS_CLASS_NAME = "resultCard"
listings = browser.find_elements(By.CLASS_NAME, f"{LISTINGS_CLASS_NAME}")
listings

[<selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="b85cc637-3060-4026-a55a-8cdc3243a9c7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="b6e69b34-ac94-4da8-95e0-feb8fc78dc61")>,
 <selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="e05b9b93-6ee7-4776-b0a9-d4eb83c6141c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="d6f985b7-6423-405c-b00e-2e2a87fe3723")>,
 <selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="c02511e4-3e33-4db5-84df-f937c7da71b5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="98a9e4f4-5a0d-4db8-bd26-bc4309fe01b1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="35034376c8ab8d7d27196c8da259d86f", element="12cc1351-6417-4803-8009-e9

In [51]:
#listings[0].find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]

'Station Wagon'

In [57]:
#listings[0].find_element(By.CLASS_NAME, "camper-details").text

'auf Anfrage\nUnbegrenzte Freimeilen\nFahrzeugversicherung mit Selbstbeteiligung USD 3.000\nZusatzversicherung zur Rückerstattung der Selbstbeteiligung\nEUR 10 Mio. Zusatz-Haftpflichtversicherung\nZusatzfahrer\nGesamtpreis\n1171,00 EUR\n68,88 EUR pro Tag'

In [65]:
#money = listings[0].find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')

In [69]:
# import re

# '.'.join(re.findall(r'\d+', money[0]))

'1171.00'

In [78]:
trip_data = []
for card in listings:

    vehicle_class = card.find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]
    company_name = card.find_element(By.CLASS_NAME, "providerLogo").get_attribute("alt")
    price_string = card.find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')
    total_price = '.'.join(re.findall(r'\d+', price_string[0]))
    daily_price = '.'.join(re.findall(r'\d+', price_string[1]))


    trip_data.append({
    'city': trip['city'],
    'start_date':trip['start_date'],
    'end_date':trip['end_date'],
    'total_price':total_price,
    'daily_price': daily_price,
    'vehicle_class':vehicle_class,
    'company_name':company_name
    })

pd.DataFrame(trip_data)

,city,start_date,end_date,total_price,daily_price,vehicle_class,company_name
0,Los Angeles,2022-11-14,2022-12-01,1171.00,68.88,Station Wagon,Travellers Autobarn
1,Los Angeles,2022-11-14,2022-12-01,1610.00,94.71,Hitop Campervan,Travellers Autobarn
2,Los Angeles,2022-11-14,2022-12-01,1614.00,94.94,Kuga Campervan,Travellers Autobarn
3,Los Angeles,2022-11-14,2022-12-01,1666.00,98.00,Motorhome C25,Road Bear
4,Los Angeles,2022-11-14,2022-12-01,1717.00,101.00,Campervan B21,Road Bear
...,...,...,...,...,...,...,...
62,Los Angeles,2022-11-14,2022-12-01,4487.00,263.94,Motorhome AF33,El Monte
63,Los Angeles,2022-11-14,2022-12-01,4708.00,276.94,Motorhome MS31,Mighty
64,Los Angeles,2022-11-14,2022-12-01,4859.00,285.82,Motorhome FS31,El Monte
65,Los Angeles,2022-11-14,2022-12-01,5100.00,300.00,Motorhome A32,Road Bear


In [74]:
LOAD_MORE_RESULTS = "document.querySelector('#ResultList > button')"

browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()